In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data_gini.csv')
print(data)

       Income     Age House
0   Very High   Young   Yes
1        High  Medium   Yes
2         Low   Young    No
3        High  Medium   Yes
4   Very High  Medium   Yes
5      Medium   Young   Yes
6        High     Old   Yes
7      Medium  Medium    No
8         Low     Old    No
9         Low   Young    No
10       High   Young   Yes
11     Medium     Old    No


In [4]:
def G(df:pd.DataFrame,target):
    counts = df[target].value_counts()
    gini = 1 - ((counts/counts.sum())**2).sum()
    return gini  

In [5]:
def cart_tree(df: pd.DataFrame, target):
    features = df.drop([target], axis=1).columns
    values = {x: df[x].unique() for x in df}

    Gini = {}

    min_feature = features[0]
    min_ai = 10

    for feature in features:
        gi = pd.Series({v: G(df[df[feature] == v], target)
                       for v in values[feature]})

        counts = [len(df[df[feature] == v]) for v in gi.index]
        ai = (gi * counts / len(df)).sum()

        Gini[feature] = gi

        if ai < min_ai:
            min_feature = feature
            min_ai = ai

    min_g = Gini[min_feature]
    return {min_feature: {v: (df[df[min_feature] == v][target].value_counts().to_dict() if min_g[v] == 0 else cart_tree(df[df[min_feature] == v].drop(columns=[min_feature]), target))
                          for v in min_g.index}}


def print_tree(tree, features, depth):
    for k in tree:
        if (type(tree[k]) == dict):
            if k in features:
                print('\t'*depth, f'[{k}]:', end=' ')
            else:
                print('\t'*depth, k, ':', end=' ')
            tkk = list(tree[k].keys())
            if (len(tkk) > 1 or tkk[0] in features):
                print()
            print_tree(tree[k], features, depth+1)
        else:
            print(k)     

In [6]:
def CART(df: pd.DataFrame, target):
    Tree = cart_tree(df, target)
    print_tree(Tree, df.drop(columns=[target]).columns, 0)

CART(data,'House')

 [Income]: 
	 Very High : Yes
	 High : Yes
	 Low : No
	 Medium : 
		 [Age]: 
			 Young : Yes
			 Medium : No
			 Old : No


In [7]:
tennis = pd.read_csv('./Tennis.csv')
CART(tennis,'Play')

 [Outlook]: 
	 Sunny : 
		 [Humidity]: 
			 High : No
			 Normal : Yes
	 Overcast : Yes
	 Rain : 
		 [Wind]: 
			 Weak : Yes
			 Strong : No


In [12]:
mushroom = pd.read_csv('./mushrooms.csv')

CART(mushroom,'class')

 [odor]: 
	 p : p
	 a : e
	 l : e
	 n : 
		 [spore-print-color]: 
			 n : e
			 k : e
			 w : 
				 [habitat]: 
					 w : e
					 l : 
						 [cap-color]: 
							 c : e
							 n : e
							 w : p
							 y : p
					 d : 
						 [gill-size]: 
							 n : p
							 b : e
					 g : e
					 p : e
			 h : e
			 r : p
			 o : e
			 y : e
			 b : e
	 f : p
	 c : p
	 y : p
	 s : p
	 m : p
